In [ ]:
#Import Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# combine all the data/usda_* into one dataframe
usda = pd.concat([pd.read_csv(f'data/usda_data_{year}.csv')  for year in range(1950, 2024)])

# combine all the data/weather_* into one dataframe
weather = pd.concat([pd.read_csv(f'data/weather_data_{year}.csv') for year in range(2000, 2021)])


In [ ]:
usda.info()

In [ ]:
# what is the average value in udsa data for each year
usda.groupby('year')['Value'].mean()


I'll select the columns that I need for analysis. I will focus on year, county_name and Value.

In [ ]:
# what is the average value in udsa data for each year
usda.groupby('year')['Value'].sum()

In [ ]:
usda['year'] = pd.to_datetime(usda['year'],
               format='%Y')
usda_subset = usda[['year','county_name','Value']]

In [ ]:
usda_subset['county_name'].nunique() #85 Michigan has 83

The result is that the data set has 85 and Michigan only has 83. The additional two are 'OTHER COUNTIES' and 'OTHER (COMBINED) COUNTIES'.

In [ ]:
usda_subset_other_combined = usda_subset[usda_subset['county_name'] == 'OTHER (COMBINED) COUNTIES']
usda_subset_other_combined

There are 144 records that contain 'OTHER (COMBINED) COUNTIES'.

In [ ]:
usda_subset_other = usda_subset[usda_subset['county_name'] == 'OTHER COUNTIES']
usda_subset_other

	year	county_name	Value
60	2020-01-01	OTHER COUNTIES	151.0
57	2021-01-01	OTHER COUNTIES	150.2
59	2022-01-01	OTHER COUNTIES	163.2
57	2023-01-01	OTHER COUNTIES	166.3

There are 4 records (years) that contain 'OTHER COUNTIES'. We will focus on the year 2020.

In [ ]:
usda_other = usda[usda['county_name'] == 'OTHER COUNTIES']
usda_other

The asd_desc is null for all 4 records. location_desc for all 4 records is "MICHIGAN, OTHER COUNTIES".

In [ ]:
usda_subset_2020=usda_subset[usda_subset['year'] == '2020-01-01']

In [ ]:
usda_subset_2020['county_name'].nunique()

There are 61 counties in the data set for 2020.

In [ ]:
usda_subset_2020['county_name'].unique()

array(['DELTA', 'DICKINSON', 'MENOMINEE', 'ANTRIM', 'BENZIE',
       'CHARLEVOIX', 'EMMET', 'GRAND TRAVERSE', 'LEELANAU', 'MANISTEE',
       'WEXFORD', 'ALCONA', 'ALPENA', 'IOSCO', 'OGEMAW', 'OTSEGO',
       'PRESQUE ISLE', 'MASON', 'MUSKEGON', 'NEWAYGO', 'OCEANA',
       'GLADWIN', 'ISABELLA', 'MECOSTA', 'MIDLAND', 'MONTCALM', 'ARENAC',
       'BAY', 'HURON', 'SAGINAW', 'SANILAC', 'TUSCOLA', 'ALLEGAN',
       'BERRIEN', 'CASS', 'KALAMAZOO', 'KENT', 'OTTAWA', 'VAN BUREN',
       'BARRY', 'BRANCH', 'CALHOUN', 'CLINTON', 'EATON', 'HILLSDALE',
       'INGHAM', 'IONIA', 'JACKSON', 'ST JOSEPH', 'SHIAWASSEE', 'GENESEE',
       'LAPEER', 'LENAWEE', 'LIVINGSTON', 'MACOMB', 'MONROE', 'OAKLAND',
       'ST CLAIR', 'WASHTENAW', 'WAYNE', 'OTHER COUNTIES'], dtype=object)

In [ ]:
import matplotlib.pyplot as plt
sns.histplot(data=usda_subset, x="Value", hue="county_name", legend=False)
plt.xlabel('BU/Acre')
plt.title('Bushels/Acre by County for Past 2 Decades')
plt.show()

In [ ]:
# Adjust figure size for better readability
plt.figure(figsize=(12, 6))

# Create the scatter plot
g = sns.scatterplot(data=usda_subset_2020, x="county_name", y="Value", hue="year")

# Rotate the x-axis labels for readability
plt.xticks(rotation=90)

# Add labels and title
plt.xlabel('Counties')
plt.ylabel('BU/Acre')
plt.title('Bushels/Acre by County for 2020')

# Adjust the legend positioning
#g.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)

# Display the plot
plt.show()

It appears that perhaps the "Other Counties" was a dumping ground for smaller quanities collected over several counties. There is only one dot respresenting the one entry for the year 2020.

Was there really 3 entries for 1960 for "Other (Combined) Counties"? A dataset containing the 1960 entries will be created and investigated.

In [ ]:
usda_subset_1960=usda_subset[usda_subset['year'] == '1960-01-01']

In [ ]:
# Adjust figure size for better readability
plt.figure(figsize=(12, 6))

# Create the scatter plot
g = sns.scatterplot(data=usda_subset_1960, x="county_name", y="Value", hue="year")

# Rotate the x-axis labels for readability
plt.xticks(rotation=90)

# Add labels and title
plt.xlabel('Counties')
plt.ylabel('BU/Acre')
plt.title('Bushels/Acre by County for 1960')

# Adjust the legend positioning
#g.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)

# Display the plot
plt.show()

Sure enough there are three dots over the "Other (Combined) Counties" mark. What does it look like when we look at the entered values?

In [ ]:
usda_1960 = usda[(usda['year'] == '1960-01-01') & (usda['county_name'] == 'OTHER (COMBINED) COUNTIES')]
usda_1960

The 3 entries for 1960 have the following asd_desc: "UPPER PENINSULA", "NORTHWEST", and "NORTHEAST". The location_desc has these entries: "MICHIGAN, UPPER PENINSULA, OTHER (COMBINED) COUNTIES", "MICHIGAN, NORTHWEST, OTHER (COMBINED) COUNTIES", and "MICHIGAN, NORTHEAST, OTHER (COMBINED) COUNTIES".

In [ ]:
grouped_df = usda_subset_1960.groupby('county_name')['Value'].sum().reset_index()
print(grouped_df)

The total Value for 1960 "OTHER (COMBINED) COUNTIES" was 130.6
A similiar dumping ground value when compared to the single "OTHER COUNTIES" in 2020.

2019 seemed to have a lot of entries for "OTHER (COMBINED) COUNTIES". What is going on?

In [ ]:
usda_2019 = usda[(usda['year'] == '2019-01-01') & (usda['county_name'] == 'OTHER (COMBINED) COUNTIES')]
usda_2019

The 7 entries for 2019 have the following asd_desc: "UPPER PENINSULA", "NORTHWEST", "NORTHEAST","WEST CENTRAL", "CENTRAL", "SOUTHWEST", and "SOUTH CENTRAL". The location_desc has these entries: "MICHIGAN, UPPER PENINSULA, OTHER (COMBINED) COUNTIES", "MICHIGAN, NORTHWEST, OTHER (COMBINED) COUNTIES", "MICHIGAN, NORTHEAST, OTHER (COMBINED) COUNTIES","MICHIGAN, WEST CENTRAL, OTHER (COMBINED) COUNTIES", "MICHIGAN, CENTRAL, OTHER (COMBINED) COUNTIES", "MICHIGAN, SOUTHWEST, OTHER (COMBINED) COUNTIES", and "MICHIGAN, SOUTH CENTRAL, OTHER (COMBINED) COUNTIES".

How many years and how many entries are we looking at that are like this? It looks like 2019 was the worst with 7 regions identified. The amount of bushels/acre is  886.3. This is considerably more than 1960.

In [ ]:
grouped_df = usda.groupby(['year', 'county_name']).agg(
    count=('county_name', 'count'),
    sum=('Value', 'sum')
).reset_index()

filtered_df = grouped_df[grouped_df['count'] > 1]
# Sort by 'sum' in descending order
filtered_df = filtered_df.sort_values(by='sum', ascending=False)

print(filtered_df)

             year                county_name  count    sum
4768 2019-01-01  OTHER (COMBINED) COUNTIES      7  886.3
4610 2016-01-01  OTHER (COMBINED) COUNTIES      6  824.1
4551 2015-01-01  OTHER (COMBINED) COUNTIES      6  823.4
4670 2017-01-01  OTHER (COMBINED) COUNTIES      5  655.8
3942 2005-01-01  OTHER (COMBINED) COUNTIES      5  576.0
3638 2000-01-01  OTHER (COMBINED) COUNTIES      6  565.0
4001 2006-01-01  OTHER (COMBINED) COUNTIES      5  555.0
3589 1999-01-01  OTHER (COMBINED) COUNTIES      5  533.0
4061 2007-01-01  OTHER (COMBINED) COUNTIES      6  502.0
3881 2004-01-01  OTHER (COMBINED) COUNTIES      5  453.0
4120 2008-01-01  OTHER (COMBINED) COUNTIES      5  442.0
4493 2014-01-01  OTHER (COMBINED) COUNTIES      4  436.4
3820 2003-01-01  OTHER (COMBINED) COUNTIES      4  398.0
4717 2018-01-01  OTHER (COMBINED) COUNTIES      3  397.1
4248 2010-01-01  OTHER (COMBINED) COUNTIES      3  385.7
3464 1997-01-01  OTHER (COMBINED) COUNTIES      4  345.0
3691 2001-01-01  OTHER (COMBINED) COUNTIES      6  327.0
3757 2002-01-01  OTHER (COMBINED) COUNTIES      3  308.0
4183 2009-01-01  OTHER (COMBINED) COUNTIES      3  303.0
4314 2011-01-01  OTHER (COMBINED) COUNTIES      3  288.7
4432 2013-01-01  OTHER (COMBINED) COUNTIES      2  245.4
4377 2012-01-01  OTHER (COMBINED) COUNTIES      3  239.2
3530 1998-01-01  OTHER (COMBINED) COUNTIES      3  230.0
3406 1996-01-01  OTHER (COMBINED) COUNTIES      3  206.0
3340 1995-01-01  OTHER (COMBINED) COUNTIES      2  180.0
1232 1967-01-01  OTHER (COMBINED) COUNTIES      3  172.8
2190 1980-01-01  OTHER (COMBINED) COUNTIES      2  160.6
1128 1965-01-01  OTHER (COMBINED) COUNTIES      3  159.3
1180 1966-01-01  OTHER (COMBINED) COUNTIES      3  152.5
2044 1978-01-01  OTHER (COMBINED) COUNTIES      2  150.0
2117 1979-01-01  OTHER (COMBINED) COUNTIES      2  147.4
952  1962-01-01  OTHER (COMBINED) COUNTIES      3  144.8
1004 1963-01-01  OTHER (COMBINED) COUNTIES      3  142.2
900  1961-01-01  OTHER (COMBINED) COUNTIES      3  139.2
848  1960-01-01  OTHER (COMBINED) COUNTIES      3  130.6

When comparing to the rest of the entries in 2019. How bad does it look?

In [ ]:
usda_subset_2019=usda_subset[usda_subset['year'] == '2019-01-01']
grouped_df_2019 = usda_subset_2019.groupby(['year', 'county_name']).agg(
    count=('county_name', 'count'),
    sum=('Value', 'sum')
).reset_index()

In [ ]:
# Adjust figure size for better readability
plt.figure(figsize=(12, 6))

# Create the scatter plot
g = sns.scatterplot(data=grouped_df_2019, x="county_name", y="sum", hue="year")

# Rotate the x-axis labels for readability
plt.xticks(rotation=90)

# Add labels and title
plt.xlabel('Counties')
plt.ylabel('BU/Acre')
plt.title('Bushels/Acre by County for 2019')

# Adjust the legend positioning
#g.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)

# Display the plot
plt.show()

The rest of the counties for 2019 produced no more than 200 bushels/acre. As expected, "OTHER (COMBINED) COUNTIES" is the highest in the chart.

In [ ]:
# GDD calculation function
def calculate_gdd(df, base_temp=50, upper_temp=86):
    """
    Calculate Growing Degree Days (GDD) for corn.
    """
    df['TMAX'] = df['TMAX'].clip(lower=base_temp, upper=upper_temp)
    df['TMIN'] = df['TMIN'].clip(lower=base_temp)
    df['TAVG'] = (df['TMAX'] + df['TMIN']) / 2
    df['GDD'] = df['TAVG'] - base_temp
    return df

In [ ]:
weather = calculate_gdd(weather)

In [ ]:
# Compare average yield for usda_data_1955 and usda_data_2015
usda_1955 = pd.read_csv('data/usda_data_1955.csv')
usda_2015 = pd.read_csv('data/usda_data_2015.csv')

usda_1955['Value'].mean(), usda_2015['Value'].mean()

# whats the std deviation of yield for those years
usda_1955['Value'].std(), usda_2015['Value'].std()


In [ ]:
# Compare total gdd for weather_data_1955 and weather_data_2015 for county_ansi=161
weather_2014 = pd.read_csv('data/weather_data_2014.csv')
weather_2023 = pd.read_csv('data/weather_data_2023.csv')

weather_2014 = calculate_gdd(weather_2014)
weather_2023 = calculate_gdd(weather_2023)

weather_2014[weather_2014['county_ansi'] == 161]['GDD'].sum(), weather_2023[weather_2023['county_ansi'] == 161]['GDD'].sum()

# Do this for all years between 1950 and 1959
gdd = []
for year in range(1950, 1960):
    weather = pd.read_csv(f'data/weather_data_{year}.csv')
    weather = calculate_gdd(weather)
    gdd.append(weather[weather['county_ansi'] == 161]['GDD'].sum())

gdd


In [ ]:
#Combine all weather data
weather_data = pd.concat([pd.read_csv(f'data/weather_data_{year}.csv') for year in range(1950, 2024)])
# Assuming 'weather_data' is your concatenated weather DataFrame for all years
weather_data = calculate_gdd(weather_data)
weather_data


In [ ]:
# Convert 'date' to datetime if it's not already
weather_data['date'] = pd.to_datetime(weather_data['date'])

# Extract year and month
weather_data['year'] = weather_data['date'].dt.year
weather_data['month'] = weather_data['date'].dt.month

# Aggregate GDD by county and year
gdd_annual = weather_data.groupby(['state_ansi', 'county_ansi', 'year'])['GDD'].sum().reset_index()


In [ ]:
# Combine all USDA data into a single DataFrame usda_data
usda_data = pd.concat([pd.read_csv(f'data/usda_data_{year}.csv') for year in range(1950, 2024)])

In [ ]:
# Filter for the relevant data (e.g., 'YIELD' in 'statisticcat_desc')
corn_data = usda_data[usda_data['statisticcat_desc'] == 'YIELD']

# Convert 'Value' to numeric, removing any commas or missing values
corn_data['Value'] = corn_data['Value'].replace(',', '', regex=True)
corn_data = corn_data[corn_data['Value'] != '(D)']  # Remove suppressed data
corn_data['Value'] = pd.to_numeric(corn_data['Value'])

# Select relevant columns
corn_data = corn_data[['state_ansi', 'county_ansi', 'year', 'Value']]


In [ ]:
# Merge on 'state_ansi', 'county_ansi', and 'year'
merged_data = pd.merge(gdd_annual, corn_data, on=['state_ansi', 'county_ansi', 'year'])
# Remove entries with zero or NaN yields
merged_data = merged_data[merged_data['Value'] > 0]
# Remove entries with zero or NaN yields
merged_data = merged_data[merged_data['GDD'] > 0]



In [ ]:
# Dump merged_data to csv
merged_data.to_csv('merged_data.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Scatter plot with regression line
sns.lmplot(x='GDD', y='Value', data=merged_data)
plt.title('Correlation between GDD and Corn Yield')
plt.xlabel('Growing Degree Days (GDD)')
plt.ylabel('Corn Yield (bu/acre)')
plt.show()


In [ ]:
# Make the same chart with PRCP instead of GDD

# Calculate total precipitation for the growing season
weather_data['PRCP'] = weather_data['PRCP'].clip(lower=0)
prcp_annual = weather_data.groupby(['state_ansi', 'county_ansi', 'year'])['PRCP'].sum().reset_index()

# Merge on 'state_ansi', 'county_ansi', and 'year'
merged_data_prcp = pd.merge(prcp_annual, corn_data, on=['state_ansi', 'county_ansi', 'year'])
# Remove entries with zero or NaN yields
merged_data_prcp = merged_data_prcp[merged_data_prcp['Value'] > 0]
# Remove entries with zero or NaN yields
merged_data_prcp = merged_data_prcp[merged_data_prcp['PRCP'] > 0]

# Calculate the Pearson correlation coefficient
correlation_prcp = merged_data_prcp['PRCP'].corr(merged_data_prcp['Value'])
print(f"Pearson correlation coefficient between PRCP and corn yield: {correlation_prcp}")

# Scatter plot with regression line
sns.lmplot(x='PRCP', y='Value', data=merged_data_prcp)
plt.title('Correlation between PRCP and Corn Yield')
plt.xlabel('Total Precipitation (mm)')
plt.ylabel('Corn Yield (bu/acre)')
plt.show()


In [ ]:
# Aggregate precipitation data
prcp_annual = weather_data.groupby(['state_ansi', 'county_ansi', 'year'])['PRCP'].sum().reset_index()
merged_data = pd.merge(merged_data, prcp_annual, on=['state_ansi', 'county_ansi', 'year'])


In [ ]:
import statsmodels.api as sm

# Define independent variables and dependent variable
X = merged_data[['GDD', 'PRCP']]
y = merged_data['Value']

# Add a constant term for the intercept
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()
print(model.summary())


Let's break down the key components of your OLS (Ordinary Least Squares) regression results:

### 1. **Model Overview:**
   - **Dep. Variable (Dependent Variable):** This is the variable you're trying to predict, in this case, labeled as "Value."
   - **Model:** OLS, meaning this is a simple linear regression model using the least squares method.
   - **No. Observations:** 4673 observations (data points) were used in this regression.

### 2. **R-squared:**
   - **R-squared (0.099):** This indicates that the model explains about **9.9% of the variance** in the dependent variable ("Value"). This is a relatively low R-squared value, meaning that most of the variability in the data is not explained by the model.
   - **Adj. R-squared (0.098):** This is a slightly adjusted version of R-squared that accounts for the number of predictors in the model. It's still low, meaning the model is not very strong at explaining the data.

### 3. **F-statistic and p-value (Prob F-statistic):**
   - **F-statistic (255.8):** This tests the overall significance of the model. A higher F-statistic generally means the model is a good fit.
   - **Prob(F-statistic) (3.88e-106):** The p-value associated with the F-statistic is incredibly small (close to zero), meaning the model is statistically significant overall. So, even though the model doesn't explain much variance (low R-squared), it's still better than having no model at all.

### 4. **Coefficients (coef), t-values, and p-values:**
   These are the estimated effects of each independent variable on the dependent variable.
   - **const (Intercept) (0.0849):** This is the value of the dependent variable ("Value") when all independent variables (GDD, PRCP) are zero. The p-value for this is 0.983, which means it's not statistically significant.
   - **GDD (0.0274):** This means that for each unit increase in GDD (an independent variable), the dependent variable increases by approximately 0.0274 units. The p-value is 0.000, so this is a statistically significant effect.
   - **PRCP (1.9657):** This means that for each unit increase in PRCP, the dependent variable increases by 1.9657 units. The p-value is also 0.000, indicating a highly significant effect.

### 5. **Standard Error (std err):**
   - The standard errors give a measure of the variability in the coefficient estimates. For example, the standard error for GDD is 0.002, which indicates a small amount of variability, suggesting a reliable estimate.

### 6. **t-value and p-value:**
   - **t-value:** This tells you how many standard errors the coefficient is away from zero. Higher absolute t-values indicate that the corresponding coefficient is statistically significant.
   - **p-value:** This tells you whether the coefficient is statistically significant. For GDD and PRCP, the p-values are 0.000, meaning both are statistically significant. The intercept is not significant (p-value = 0.983).

### 7. **Durbin-Watson (0.422):**
   - This statistic tests for autocorrelation in the residuals (errors). A value close to 2 means there is no autocorrelation. Here, 0.422 is quite low, suggesting the possibility of positive autocorrelation, which might need further investigation.

### 8. **Omnibus and Jarque-Bera (JB) tests:**
   - These are tests for normality of the residuals. The p-values for these tests are very small (Prob(Omnibus) = 0.000, Prob(JB) = 8.09e-35), indicating that the residuals are not normally distributed, which could be a problem depending on the assumptions of your model.

### 9. **Condition Number (1.66e+04):**
   - A high condition number suggests multicollinearity, meaning that the independent variables are highly correlated. The condition number here (16,600) is quite high, indicating potential multicollinearity issues.

### Conclusion:
- The model is statistically significant, but it doesn't explain much of the variability in the dependent variable (low R-squared).
- The variables **GDD** and **PRCP** have a significant impact on the dependent variable ("Value").
- However, there may be issues with autocorrelation (Durbin-Watson) and multicollinearity (high condition number), and the residuals don't appear to be normally distributed (Omnibus and JB tests). You might need to address these issues if you want to improve the model.

In [ ]:
# Assuming usda_data is your concatenated USDA DataFrame for all years
# Ensure 'Value' is cleaned and converted to numeric
usda_data['Value'] = usda_data['Value'].replace(',', '', regex=True)
usda_data['Value'] = usda_data['Value'].replace({'(D)': np.nan, '(Z)': np.nan, '(NA)': np.nan, '': np.nan})
usda_data['Value'] = pd.to_numeric(usda_data['Value'], errors='coerce')

# Filter for the relevant data (e.g., 'YIELD' in 'statisticcat_desc')
corn_data = usda_data[usda_data['statisticcat_desc'] == 'YIELD']

# Select relevant columns, including 'county_name'
corn_data = corn_data[['state_ansi', 'county_ansi', 'year', 'Value', 'county_name']]

# Drop rows with missing values in 'Value' or 'county_name'
corn_data.dropna(subset=['Value', 'county_name'], inplace=True)

# Ensure 'year' is integer
corn_data['year'] = corn_data['year'].astype(int)


In [ ]:
# Merge on 'state_ansi', 'county_ansi', and 'year'
merged_data = pd.merge(gdd_annual, corn_data, on=['state_ansi', 'county_ansi', 'year'])

# Optional: Drop any rows with missing values
merged_data.dropna(subset=['GDD', 'Value'], inplace=True)


In [ ]:
# Calculate average yield per county
avg_yield_per_county = merged_data.groupby(['county_ansi', 'county_name'])['Value'].mean().reset_index()

# Sort and select top 10 counties
top_counties = avg_yield_per_county.sort_values(by='Value', ascending=False).head(20)

# Filter data for these counties
top_county_codes = top_counties['county_ansi'].unique()
top_county_names = top_counties['county_name'].unique()
top_counties_data = merged_data[merged_data['county_ansi'].isin(top_county_codes)]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set up the plotting environment
sns.set_style('whitegrid')
num_counties = len(top_county_codes)
fig, axes = plt.subplots(nrows=num_counties, ncols=1, figsize=(12, num_counties * 4), sharex=True)

# Ensure axes is iterable
if num_counties == 1:
    axes = [axes]

for ax, county_code, county_name in zip(axes, top_county_codes, top_county_names):
    county_data = top_counties_data[top_counties_data['county_ansi'] == county_code]
    county_data = county_data.sort_values('year')

    # Plot Yield
    ax.plot(county_data['year'], county_data['Value'], color='blue', marker='o', label='Corn Yield (bu/acre)')
    ax.set_ylabel('Corn Yield (bu/acre)', color='blue')
    ax.tick_params(axis='y', labelcolor='blue')

    # Create a second y-axis for GDD
    ax2 = ax.twinx()
    ax2.plot(county_data['year'], county_data['GDD'], color='green', marker='x', label='GDD')
    ax2.set_ylabel('GDD', color='green')
    ax2.tick_params(axis='y', labelcolor='green')

    # Title with county name
    ax.set_title(f'Corn Yield and GDD Over Time - {county_name}')

    # Add legends
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(lines + lines2, labels + labels2, loc='upper left')

# Set common x-label
plt.xlabel('Year')
plt.tight_layout()
plt.show()
